# 定期券買ったほうがいいのかな？
在宅勤務が多かったうちは回数券で安く済ませてたけど、最近出勤する用事が増えてしまって交通費の出費が厳しくなってきた。<br>
ので、向こう半年のうち何日出勤するようならば定期券を買うべきなのか損益分岐点を求めてみる。<br>
わざわざNotebookにしたのはGitHubにプッシュするネタが欲しかったからです。

### 参考にした記事・サイト
- https://holidays-jp.github.io/
- https://qiita.com/dkugi/items/8c32cc481b365c277ec2
- https://qiita.com/calderarie/items/0ef921b476911a55148d
- https://www.techscore.com/blog/2019/12/16/better-way-handling-json-data-in-python/
- https://techacademy.jp/magazine/15826

In [1]:
import json, datetime, requests, pickle
from datetime import timedelta
from dateutil.relativedelta import relativedelta

%matplotlib inline
from matplotlib import pyplot

from_ = datetime.date.today()
until_ = from_ + relativedelta(months=6)

# 消化予定の有給
paid_holiday = 8

# 回数券 vs 定期券
coupon_tickets = 2800 / 11 * 2
six_month_commuter_pass = 53950

In [2]:
def dateRange(from_: datetime, until_:datetime):
    diff = (until_ - from_).days + 1
    return (from_ + timedelta(i) for i in range(diff))

In [3]:
def isHoliday(date: datetime):
    date = str(date)
    
    try:
        with open('holidays.pkl', 'rb') as f:
            holidays = pickle.load(f)       
    except EOFError:
        url = 'https://holidays-jp.github.io/api/v1/date.json'
        response = requests.get(url)
        holidays = response.json()
        with open('holidays.pkl', 'wb') as f:
            pickle.dump(holidays, f)
   
    return True if holidays.get(date) != None else False

In [4]:
def getWorkdays(from_: datetime, until_:datetime):
    return sum([not isHoliday(date) for date in dateRange(from_, until_)])

In [5]:
def getCost(workdays, cost):
    return workdays * cost

In [6]:
def main():
    workdays = getWorkdays(from_, until_) - paid_holiday
    n_weeks = ((until_ - from_) / 7).days
    
    for wfh_freq in [0.5, 1, 1.5, 2, 2.5, 3]: # per week
        wfo_days = workdays - int(n_weeks * wfh_freq)
        print('◆週{:}日在宅勤務を行う場合：'.format(wfh_freq))
        print('\t回数券：{:.1f}円'.format(wfo_days * coupon_tickets))
        print('\t定期券：{:}円'.format(six_month_commuter_pass))
        print('\t')

In [7]:
main()

◆週0.5日在宅勤務を行う場合：
	回数券：78909.1円
	定期券：53950円
	
◆週1日在宅勤務を行う場合：
	回数券：72290.9円
	定期券：53950円
	
◆週1.5日在宅勤務を行う場合：
	回数券：65672.7円
	定期券：53950円
	
◆週2日在宅勤務を行う場合：
	回数券：59054.5円
	定期券：53950円
	
◆週2.5日在宅勤務を行う場合：
	回数券：52436.4円
	定期券：53950円
	
◆週3日在宅勤務を行う場合：
	回数券：45818.2円
	定期券：53950円
	


# 結論
私の場合、だいたい週2.5日が分岐点で、月の半分以上在宅勤務すれば定期券を買わなくとも元が取れそうでした。<br>
そして今のペースだと赤字になることが判明したので今度定期買うことにしました。